In [8]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
from deepface import DeepFace
import mediapipe as mp
from scipy.spatial import distance as dist
import imutils

mp_face = mp.solutions.face_mesh
face_mesh = mp_face.FaceMesh(static_image_mode=False,
                             max_num_faces=10,
                             refine_landmarks=True,
                             min_detection_confidence=0.5)

EAR_THRESH = 0.22       
EAR_CONSEC_FRAMES = 15  


LEFT_EYE  = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]

I0000 00:00:1758550671.240674   36879 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1758550671.244339   38090 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.2.2-arch1.2), renderer: Mesa Intel(R) Iris(R) Xe Graphics (TGL GT2)
W0000 00:00:1758550671.246450   38085 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [9]:
def calc_ear(landmarks, eye_idx, w, h):
    pts = [(int(landmarks[i].x * w), int(landmarks[i].y * h)) for i in eye_idx]
    A = dist.euclidean(pts[1], pts[5])
    B = dist.euclidean(pts[2], pts[4])
    C = dist.euclidean(pts[0], pts[3])
    return (A + B) / (2.0 * C)

# -------------------- Tkinter GUI --------------------
root = tk.Tk()
root.title("Drowsiness Detection System")
root.geometry("1000x700")

panel = tk.Label(root)
panel.pack()

def analyze_frame(frame, counter_map):
    """
    Returns:
        annotated_frame,
        sleeping_count,
        list_of_sleeping_ages
    """
    h, w = frame.shape[:2]
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = face_mesh.process(rgb)

    sleeping_count = 0
    sleeping_ages = []

    if result.multi_face_landmarks:
        for idx, face in enumerate(result.multi_face_landmarks):
            leftEAR = calc_ear(face.landmark, LEFT_EYE, w, h)
            rightEAR = calc_ear(face.landmark, RIGHT_EYE, w, h)
            ear_val = (leftEAR + rightEAR) / 2.0

            xs = [int(l.x * w) for l in face.landmark]
            ys = [int(l.y * h) for l in face.landmark]
            x1, y1, x2, y2 = min(xs), min(ys), max(xs), max(ys)

            if idx not in counter_map:
                counter_map[idx] = 0

            if ear_val < EAR_THRESH:
                counter_map[idx] += 1
            else:
                counter_map[idx] = 0

            # Determine status
            if counter_map[idx] >= EAR_CONSEC_FRAMES:
                status = "Sleeping"
                color = (0, 0, 255)
                sleeping_count += 1

                # Age prediction (do every ~30 frames to save time)
                if counter_map[idx] % 30 == 0:
                    try:
                        crop = frame[max(0,y1):min(h,y2), max(0,x1):min(w,x2)]
                        pred = DeepFace.analyze(crop, actions=['age'],
                                                enforce_detection=False)
                        age_val = str(int(pred['age']))
                        sleeping_ages.append(age_val)
                    except:
                        sleeping_ages.append("Unknown")
            else:
                status = "Awake"
                color = (0, 255, 0)

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, status, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    return frame, sleeping_count, sleeping_ages

def show_popup(count, ages):
    if count > 0:
        messagebox.showinfo("Alert",
            f"Sleeping People: {count}\nAges: {', '.join(ages) if ages else 'Predicting...'}")
    else:
        messagebox.showinfo("Info", "No one is sleeping.")

def process_image():
    path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.png *.jpeg")])
    if not path:
        return
    img = cv2.imread(path)
    img = imutils.resize(img, width=800)
    counter_map = {}
    processed, count, ages = analyze_frame(img, counter_map)
    processed = cv2.cvtColor(processed, cv2.COLOR_BGR2RGB)
    processed = ImageTk.PhotoImage(Image.fromarray(processed))
    panel.configure(image=processed)
    panel.image = processed
    show_popup(count, ages)


W0000 00:00:1758550671.265709   38084 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [10]:
def process_video():
    path = filedialog.askopenfilename(filetypes=[("Video files", "*.mp4 *.avi *.mov")])
    if not path:
        return
    cap = cv2.VideoCapture(path)
    counter_map = {}

    def loop():
        ret, frame = cap.read()
        if ret:
            frame = imutils.resize(frame, width=800)
            processed, count, ages = analyze_frame(frame, counter_map)
            rgb = cv2.cvtColor(processed, cv2.COLOR_BGR2RGB)
            img = ImageTk.PhotoImage(Image.fromarray(rgb))
            panel.configure(image=img)
            panel.image = img
            root.after(30, loop)
            if count > 0:
                show_popup(count, ages)
        else:
            cap.release()
    loop()

In [11]:
btn_frame = tk.Frame(root)
btn_frame.pack(pady=20)
tk.Button(btn_frame, text="Process Image", command=process_image,
          bg="#4CAF50", fg="white", width=20).grid(row=0, column=0, padx=10)
tk.Button(btn_frame, text="Process Video", command=process_video,
          bg="#2196F3", fg="white", width=20).grid(row=0, column=1, padx=10)

root.mainloop()